論文<br>
https://arxiv.org/abs/2205.08535<br>
<br>
GitHub<br>
https://github.com/hongfz16/AvatarCLIP<br>
<br>
<a href="https://colab.research.google.com/github/kaz12tech/ai_demos/blob/master/AvatarCLIP_demo.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 環境セットアップ

## GPU確認

In [ ]:
!nvidia-smi

## GitHubからコード取得

In [ ]:
%cd /content

!git clone https://github.com/hongfz16/AvatarCLIP.git
!git clone https://github.com/hongfz16/neural_renderer.git

## ライブラリのインストール

In [ ]:
%cd /content/AvatarCLIP

# Install Pytorch
!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html
# Install lib
!pip install -r requirements.txt

## neural_rendererファイルの修正

In [ ]:
%cd /content/neural_renderer/neural_renderer

with open('./perspective.py', 'r') as f:
  lines = f.readlines()
lines.insert(19, '    x[z<=0] = 0\n')
lines.insert(20, '    y[z<=0] = 0\n')
lines.insert(21, '    z[z<=0] = 0\n')
with open('./perspective.py', 'w') as f:
  for l in lines:
    f.write(l)

## neural_rendererのコンパイル

In [ ]:
%cd /content/neural_renderer

!python3 setup.py install

## SAMLのダウンロード

{SAML}(https://smpl.is.tue.mpg.de/index.html)に登録<br>
Download version 1.1.0 for Python 2.7 (female/male/neutral, 300 shape PCs)<br>をダウンロード後、Google Driveにアップロード


In [ ]:
%cd /content/AvatarCLIP

from google.colab import drive
drive.mount('/content/gdrive')

# Google DriveのファイルをGoogle Colabにコピー
!mkdir -p ./smpl_models/smpl
!cp "/content/gdrive/MyDrive/Colab Notebooks/SMPL_python_v.1.1.0.zip" "./smpl_models/smpl/SMPL_python_v.1.1.0.zip"
!unzip -d ./smpl_models/smpl/ ./smpl_models/smpl/SMPL_python_v.1.1.0.zip
!cp ./smpl_models/smpl/SMPL_python_v.1.1.0/smpl/models/basicmodel_neutral_lbs_10_207_0_v1.1.0.pkl ./smpl_models/smpl/SMPL_NEUTRAL.pkl

In [ ]:
raise Exception("Please restart runtime")

## ライブラリのインポート
以降のセル実行前に<br>
ランタイム→ランタイムの再起動


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cd /content/AvatarCLIP/AvatarGen/AppearanceGen
import os
import time
import logging
import argparse
import random
import numpy as np
import cv2 as cv
import trimesh
import torch
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.tensorboard import SummaryWriter
from shutil import copyfile
from icecream import ic
from tqdm import tqdm
from pyhocon import ConfigFactory
from models.dataset import Dataset
from models.dataset import SMPL_Dataset
from models.fields import RenderingNetwork, SDFNetwork, SingleVarianceNetwork, NeRF
from models.renderer import NeuSRenderer
from models.utils import lookat, random_eye, random_at, render_one_batch, batch_rodrigues
from models.utils import sphere_coord, random_eye_normal, rgb2hsv, differentiable_histogram
from models.utils import my_lbs, readOBJ
import clip
from smplx import build_layer
import imageio
import glob

import os
from tqdm import tqdm
import numpy as np
from IPython import display
from PIL import Image
from base64 import b64encode

to8b = lambda x : (255*np.clip(x,0,1)).astype(np.uint8)
from main import Runner

# AppearanceDescription設定

In [ ]:
AppearanceDescription = "Fat Wizard" #@param {type:"string"}

torch.set_default_tensor_type('torch.cuda.FloatTensor')
FORMAT = "[%(filename)s:%(lineno)s - %(funcName)20s() ] %(message)s"
logging.basicConfig(level=logging.INFO, format=FORMAT)
conf_path = 'confs/examples_small/example.conf'
f = open(conf_path)
conf_text = f.read()
f.close()
conf_text = conf_text.replace('{TOREPLACE}', AppearanceDescription)

#@markdown google driveに保存する場合はTrue
save_gdrive = True #@param {type:"boolean"}
#@markdown save_gdrive指定時の出力先
custom_exp_dir = "/content/gdrive/MyDrive/Colab Notebooks" #@param {type:"string"}
if save_gdrive:
  conf_text = conf_text.replace('base_exp_dir = ./exp/smpl/example', 'base_exp_dir = ' + custom_exp_dir + '/exp/smpl/example')


# print(conf_text)
conf = ConfigFactory.parse_string(conf_text)
print("Prompt: {}".format(conf.get_string('clip.prompt')))
print("Face Prompt: {}".format(conf.get_string('clip.face_prompt')))
print("Back Prompt: {}".format(conf.get_string('clip.back_prompt')))

# Avatar Generation

In [ ]:
%%time
runner = Runner(conf_path, 'train_clip', 'smpl', False, True, conf)
runner.init_clip()
runner.init_smpl()
runner.train_clip()

# 最適化プロセスのビデオ生成

In [ ]:
image_folder = 'exp/smpl/example/validations_extra_fine'
video_path = 'video.mp4'
if save_gdrive:
  image_folder = os.path.join(custom_exp_dir, image_folder)
  video_path = os.path.join(custom_exp_dir, video_path)
image_fname = glob.glob(os.path.join(image_folder, "*.png"))

init_frame = 1
last_frame = len(image_fname)
min_fps = 10
max_fps = 60
total_frames = last_frame - init_frame

length = 15 #Desired time of the video in seconds

frames = []
for i in range(init_frame, last_frame): #
    frames.append(Image.open(image_fname[i]))

#fps = last_frame/10
fps = np.clip(total_frames/length,min_fps,max_fps)

from subprocess import Popen, PIPE
p = Popen(['ffmpeg', '-y', '-f', 'image2pipe', '-vcodec', 'png', '-r', str(fps), '-i', '-', '-vcodec', 'libx264', '-r', str(fps), '-pix_fmt', 'yuv420p', '-crf', '17', '-preset', 'veryslow', video_path], stdin=PIPE)
for im in tqdm(frames):
    im.save(p.stdin, 'PNG')
p.stdin.close()
p.wait()
mp4 = open(video_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

display.HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)